# Diccionario de la Data de Excel

## Del Excel "Tratatamiento de Datos"

- **INGRESO:** ID de Paciente con Digito Verificador
- **ID_INGRESO:** ID de Paciente sin Digito Verificador
- **ELEC_URG:** Dice si es Intervencion de Urgencia o Programada (ELEC, URG)
- **AMB_HOSP:** Dice si el Paciente requiere Hospitalizacion o si es Ambulatorio (HOSP, AMB)
- **DHING:** Fecha y Hora de Ingreso del Paciente
- **DHEG:** Fecha y Hora de Egreso de Paciente
- **HRSEST:** Horas entre el Ingreso y Egreso
- **HRSESTPOSTPAB:** Horas de Post Operatorio del Paciente
- **INIPAB:** Fecha y Hora de Inicio de la Cirugua
- **FINPAB:** Fecha y Hora de la Finalizacion de la Cirugia
- **DURPAB:** Duracion de la Cirugia en Horas
- **CIRUJANO:** Nombre del Cirujano a cargo del Paciente
- **STAFF:** Dice si el CIRUJANO pertenece al Staff o no (S, N)
- **ESPECIALIDAD_MEDICO:** Especialidad del Medico
- **COMUNA:** Comuna de Origen del Paciente
- **EDAD:** Edad en años del Paciente
- **ASEGURADOR:** Asegurador o Prevision del Paciente
- **IPLAN:** Plan de Isapre o Convenio del Paciente
- **CODCIRUGIA:** Codigo Interno de la Cirugia Principal a la que sometio el Paciente
- **CIRUGIA:** Nombre de la Cirugia Principal a la que se sometio el Paciente
- **ES_PQ:** Dice si la Cirugia es Paquetizada o no (PQ, AB)
- **IND_COBRO_PAQUETE:**
- **COD_PAQUETE:** Codigo Interno del Paquete de la Cirugia
- **TIPO_TECNICA:** Codigo Interno del Tipo de Tecnica Utilizado en la Cirugia
- **DESCRIPCION:** Descripcion de la Tecnica Utilizada en la Cirugia
- **COD_CIRUGIAS:** Codigos Internos de Todas las Cirugias Realizadas al Paciente en Pabellon
- **Nombre_CIRUJANO:** Nombre Completo del Cirujano a Cargo del Paciente
- **Cantidad_Cirugias:** Numero de Cirugias Hechas al Paciente en Pabellon

**Del Excel Anterior Solo se Utilizaran:**
- INGRESO
- ID_INGRESO
- ELEC_URG
- AMB_HOSP
- DURPAB
- STAFF
- ESPECIALIDAD_MEDICO
- COMUNA
- EDAD
- ASEGURADOR
- IPLAN
- COD_CIRUGIAS
- Nombre_CIRUJANO
- Cantidad_Cirugias

## Del los Excel de "CONSUMOS"

En estos Excel's estan los datos de los insumos utilizados por paciente en cada cirugia desde el 2022 hasta el 2023, cada Excel tiene los datos de un trimestre, por lo tanto hay un total de 8 Archivos con Insumos por paciente.

- **ID_INGRESO:** ID de Paciente sin Digito Verificador
- **ID_LIQUIDACION:** Codigo Interno...(?)
- **DV_ID_INGRESO:** Numero Verificador del ID del Paciente
- **DESC_ITEM:** Area a la que Pertenece el Insumo
- **FECHA_CONTABLE:** Fecha y Hora en la que se registro el Insumo
- **FECHA_CONSUMO:** Fecha y Hora en la que se hizo uso del Insumo
- **ID_CONSUMO:** ID interno del Insumo
- **CODIGO_MOVIMIENTO:** Codigo Interno...(?)
- **TIPO_CONSUMO:** Categoria General en la que se dio uso el Insumo (EXA, RECETA)
- **COD_CENTRO:** Codigo Interno del Centro Medico (?)
- **COD_ISAPRE:** Codigo Interno de....(?)
- **NOMBRE_GRUPO_ABA:** Nombre de la Categoria del Insumo
- **NOMBRE_TIPO_ABA:** Nombre de la Sub Categoria del Insumo
- **COD_PRODUCTO:** Codigo Interno del Insumo (Solo para los Insumos del tipo 'RECETA')
- **COD_PRESTACION:** Codigo Interno de....(?)
- **NOMBRE:** Nombre del Insumo
- **NOMBRE_EMBALAJE:** Formato y Unidad de Medida del Insumo
- **AFECTO:** (?)
- **EXENTO:** (?)
- **IVA:** (?)
- **TOTAL:** (?)
- **CANTIDAD:** (?)
- **COSTO:** (?)
- **RECARGO:** (?)


**De los Excel's anteriores se utilizarán solamente los siguientes atributos:**
- ID_INGRESO
- DV_ID_INGRESO
- DESC_ITEM
- ID_CONSUMO
- TIPO_CONSUMO
- NOMBRE_GRUPO_ABA
- NOMBRE_TIPO_ABA
- NOMBRE
- NOMBRE_EMBALAJE


***Se Utilizará ID_INGRESO para el cruce de todos los archivos**

# Importar Archivos EXCEL

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Removes the limit for the number of displayed columns
pd.set_option("display.max_columns", None)

# Removes the limit for the number of displayed rows
pd.set_option("display.max_rows", None)

# To build models for prediction
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor

# To encode categorical variables
from sklearn.preprocessing import LabelEncoder

# For tuning the model
from sklearn.model_selection import GridSearchCV

# To check model performance
from sklearn.metrics import make_scorer,mean_squared_error, r2_score, mean_absolute_error

In [2]:
#Importar Tratamiento de Datos
Trat_Dat = pd.read_excel('Tratamiento de Datos.xlsx', sheet_name= "Tabla Consolidada Filtrada")

#Extraer Solo las Columnas deseadas:
columnas_deseadas = ["INGRESO", "ID_INGRESO", "ELEC_URG", "AMB_HOSP", "DURPAB", "STAFF", "ESPECIALIDAD_MEDICO", "COMUNA", "EDAD", "ASEGURADOR", "IPLAN", "COD_CIRUGIAS", "Nombre_CIRUJANO", "Cantidad_Cirugias"]

# Seleccionar solo las columnas deseadas de Trat_Dat
Trat_Dat_Final = Trat_Dat.loc[:, columnas_deseadas]


In [3]:
#Importar "COSTOS"

archivos = ['01-03 22.xlsx', '04-06 22.xlsx', '07-09 22.xlsx', '10-12 22.xlsx',
            '01-03 23.xlsx', '04-06 23.xlsx', '07-09 23.xlsx', '10-12 23.xlsx']

dataframes_concatenados = {}

for archivo in archivos:
    excel_file = pd.ExcelFile(archivo)
    hojas = excel_file.sheet_names[:-1]  # Excluir la última hoja
    dfs = []
    for hoja in hojas:
        df = pd.read_excel(excel_file, sheet_name=hoja)
        dfs.append(df)
    dataframes_concatenados[archivo] = pd.concat(dfs, ignore_index=True)
    excel_file.close()

tabla_final = pd.concat(dataframes_concatenados.values(), ignore_index=True)

In [4]:
#tabla_final.head(5) #Comprobacion de la consolidacion de los archivos

In [5]:
#Extraer Solo las Columnas deseadas:
columnas_deseadas1 = ["ID_INGRESO", "DV_ID_INGRESO", "DESC_ITEM", "ID_CONSUMO", 
                     "TIPO_CONSUMO", "NOMBRE_GRUPO_ABA", "NOMBRE_TIPO_ABA", 
                     "NOMBRE", "NOMBRE_EMBALAJE"]

# Seleccionar solo las columnas deseadas de tabla_final
Costos_Final = tabla_final.loc[:, columnas_deseadas1]


In [6]:
#Cruce entre los Dataframes de Consumos y Datos Pacientes de "Tratamiento de Datos"
Cruce_Final = pd.merge(Trat_Dat_Final, Costos_Final, on="ID_INGRESO", how="inner")

# Guardar el DataFrame como un archivo Excel
#Cruce_Final_Ex = "Cruce_Final_Ex.xlsx"
#Cruce_Final.to_excel(Cruce_Final_Ex, index=False) #NO SE PUDO POR EXCEDER EL NUMERO MAX DE FILAS (7.112.622, 22)

ValueError: This sheet is too large! Your sheet size is: 7112622, 22 Max sheet size is: 1048576, 16384

In [7]:
# Guarda el DataFrame como un archivo CSV

Cruce_Final_Ex = "Cruce_Final_Ex.csv"

Cruce_Final.to_csv(Cruce_Final_Ex, index=False)


In [12]:
print(Cruce_Final.columns)

Index(['INGRESO', 'ID_INGRESO', 'ELEC_URG', 'AMB_HOSP', 'DURPAB', 'STAFF',
       'ESPECIALIDAD_MEDICO', 'COMUNA', 'EDAD', 'ASEGURADOR', 'IPLAN',
       'COD_CIRUGIAS', 'Nombre_CIRUJANO', 'Cantidad_Cirugias', 'DV_ID_INGRESO',
       'DESC_ITEM', 'ID_CONSUMO', 'TIPO_CONSUMO', 'NOMBRE_GRUPO_ABA',
       'NOMBRE_TIPO_ABA', 'NOMBRE', 'NOMBRE_EMBALAJE'],
      dtype='object')


In [13]:
# Sacando los datos de "Cruce_Final"para definir el top 10 de 'COD_CIRUGIAS' y su correspondiente promedio, min, max y devest de "DURPAB"

# Top 10 'COD_CIRUGIAS' más frecuentes
top_10_cirugias = Cruce_Final['COD_CIRUGIAS'].value_counts().head(10).index.tolist()

# Filtro del DataFrame original para obtener solo las filas del top 10 cirugías más frecuentes
datos_top_10 = Cruce_Final[Cruce_Final['COD_CIRUGIAS'].isin(top_10_cirugias)]

# Promedio, mínimo, máximo y la desviación estándar de 'DURPAB' para estas top 10 cirugías
top_10_stats = datos_top_10.groupby('COD_CIRUGIAS')['DURPAB'].agg(['mean', 'min', 'max', 'std']).reset_index()

# Cambiando el nombre de las columnas con sus respectivos estadisticos
top_10_stats.columns = ['COD_CIRUGIAS', 'Promedio_DURPAB', 'Min_DURPAB', 'Max_DURPAB', 'Desviacion_Estandar_DURPAB']

# Guardado de resultados en un DataFrame 'Top_10_Cirugias'
Top_10_Cirugias = top_10_stats.copy()

# Mostramos el DataFrame para probarlo:
print(Top_10_Cirugias)

                                        COD_CIRUGIAS  Promedio_DURPAB  \
0          11-03-011-00|11-03-015-00|11-03-005-00|-1        74.615385   
1                              11-03-026-00|-1|-1|-1       234.410905   
2                              11-03-042-00|-1|-1|-1       107.283123   
3                    11-03-043-00|11-03-043-00|-1|-1       137.611225   
4                              17-50-061-00|-1|-1|-1       108.846154   
5                              17-50-062-00|-1|-1|-1       165.750000   
6  17-50-207-00|11-50-027-12|17-50-210-00|17-50-2...       102.222222   
7                              18-02-001-00|-1|-1|-1        62.899734   
8                              18-02-081-00|-1|-1|-1        63.338527   
9                              18-50-004-00|-1|-1|-1        91.521344   

   Min_DURPAB  Max_DURPAB  Desviacion_Estandar_DURPAB  
0          14         150                   45.885441  
1          30         480                  161.841752  
2          27         148   

In [ ]:
#Para un grafico bonito:

from tabulate import tabulate

# Supongamos que tienes un DataFrame llamado 'datos' que contiene la información de las cirugías
# y sus duraciones, y que 'COD_CIRUGIAS' es una columna en ese DataFrame.

# Primero, calculamos las 10 'COD_CIRUGIAS' más frecuentes
top_10_cirugias = datos['COD_CIRUGIAS'].value_counts().head(10).index.tolist()

# Luego, filtramos el DataFrame original para obtener solo las filas correspondientes a las 10 cirugías más frecuentes
datos_top_10 = datos[datos['COD_CIRUGIAS'].isin(top_10_cirugias)]

# Calculamos el promedio, el valor mínimo, máximo y la desviación estándar de 'DURPAB' para estas cirugías
top_10_stats = datos_top_10.groupby('COD_CIRUGIAS')['DURPAB'].agg(['mean', 'min', 'max', 'std']).reset_index()

# Cambiamos el nombre de las columnas para mayor claridad
top_10_stats.columns = ['COD_CIRUGIAS', 'Promedio_DURPAB', 'Min_DURPAB', 'Max_DURPAB', 'Desviacion_Estandar_DURPAB']

# Ordenamos los datos por 'Promedio_DURPAB' en orden descendente
top_10_stats = top_10_stats.sort_values(by='Promedio_DURPAB', ascending=False)

# Mostramos los resultados usando tabulate
print(tabulate(top_10_stats, headers='keys', tablefmt='pretty', showindex=False))

